In [10]:
# STEP 1: Install dependencies
!pip install -q openai-whisper transformers torch tqdm ffmpeg-python

import os, glob, re
import whisper
from datetime import timedelta
from transformers import pipeline
from tqdm.notebook import tqdm
import ffmpeg

# ====================================
# STEP 2: Load and Convert Audio to WAV
# ====================================
# TODO: Replace 'path/to/your/audio.mp3' with the actual path to your uploaded audio file
input_file = '/content/Sales Call example 1 - Moduslinktube.mp3'  # Change .mp3 to the actual extension of your file
output_file = 'call.wav'

# Ensure the input audio file exists
assert os.path.exists(input_file), f"❌ Input audio file not found at: {input_file}"
print("✅ Input audio found at:", input_file)

print("🎵 Converting audio to WAV...")
try:
    (
        ffmpeg
        .input(input_file)
        .output(output_file, ar=16000, ac=1)  # mono, 16kHz
        .overwrite_output()
        .run(quiet=True)
    )
    assert os.path.exists(output_file), "❌ Audio conversion failed!"
    print("✅ Audio ready as:", output_file)
except ffmpeg.Error as e:
    print(f"❌ Error during ffmpeg conversion: {e.stderr.decode()}")
    assert False, "Audio conversion failed."


# ====================================
# STEP 3: Load Models
# ====================================
whisper_model = whisper.load_model("tiny")  # fast in free Colab
sentiment_model = pipeline("sentiment-analysis")

# ====================================
# STEP 4: Transcribe
# ====================================
print("⏳ Transcribing...")
raw_result = whisper_model.transcribe(output_file, verbose=False)

segments = []
for seg in tqdm(raw_result["segments"], desc="Processing segments"):
    segments.append(seg)

text = raw_result["text"]

# Assign speakers alternately (simple heuristic)
for i, seg in enumerate(segments):
    seg["speaker"] = "sales_rep" if i % 2 == 0 else "customer"

# ====================================
# STEP 5: Analysis Functions
# ====================================
def get_talk_time_ratio(segments):
    durations = {}
    for seg in segments:
        spk = seg["speaker"]
        durations[spk] = durations.get(spk, 0) + (seg["end"] - seg["start"])
    total = sum(durations.values())
    return {s: f"{round((d/total)*100,1)}%" for s, d in durations.items()}

def count_questions(text):
    return len(re.findall(r"\?", text))

def get_longest_monologue(segments):
    longest, speaker = 0, None
    for seg in segments:
        dur = seg["end"] - seg["start"]
        if dur > longest:
            longest, speaker = dur, seg["speaker"]
    return f"{str(timedelta(seconds=int(longest)))} ({speaker})"

def analyze_sentiment(text):
    result = sentiment_model(text[:512])  # truncate for speed
    return result[0]["label"]

def actionable_insight(text):
    if "price" in text.lower():
        return "Customer mentioned pricing — follow up with a proposal."
    elif count_questions(text) < 3:
        return "Sales rep should ask more open-ended questions."
    elif "timeline" in text.lower():
        return "Customer asked about timeline — send a project plan."
    else:
        return "Encourage the sales rep to listen more and reduce monologues."

# ====================================
# STEP 6: Run Analysis
# ====================================
analysis = {
    "talk_time_ratio": get_talk_time_ratio(segments),
    "questions_asked": count_questions(text),
    "longest_monologue": get_longest_monologue(segments),
    "call_sentiment": analyze_sentiment(text),
    "actionable_insight": actionable_insight(text)
}

print("\n📊 Final Sales Call Analysis:\n")
print(analysis)

✅ Input audio found at: /content/Sales Call example 1 - Moduslinktube.mp3
🎵 Converting audio to WAV...
✅ Audio ready as: call.wav


100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 139MiB/s]
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


⏳ Transcribing...
Detected language: English


100%|██████████| 12273/12273 [00:17<00:00, 703.81frames/s]


Processing segments:   0%|          | 0/26 [00:00<?, ?it/s]


📊 Final Sales Call Analysis:

{'talk_time_ratio': {'sales_rep': '54.1%', 'customer': '45.9%'}, 'questions_asked': 8, 'longest_monologue': '0:00:11 (sales_rep)', 'call_sentiment': 'POSITIVE', 'actionable_insight': 'Customer mentioned pricing — follow up with a proposal.'}


In [17]:
# ====================================
# STEP 7:Interactive Dashboard
# ====================================
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

# --- Title ---
display(Markdown("## 🎧 Sales Call Analysis Dashboard"))
display(Markdown("---"))

# --- Talk-Time Ratio Pie Chart ---
fig1 = go.Figure(data=[go.Pie(
    labels=list(analysis["talk_time_ratio"].keys()),
    values=[float(v.strip("%")) for v in analysis["talk_time_ratio"].values()],
    hole=0.4,
    marker=dict(colors=["#4CAF50", "#2196F3"]),
    textinfo="label+percent",
    pull=[0.05, 0]
)])
fig1.update_layout(title="🗣️ Talk-Time Ratio", title_x=0.5)
fig1.show()

# --- Questions Asked (Gauge Chart) ---
fig2 = go.Figure(go.Indicator(
    mode="gauge+number",
    value=analysis["questions_asked"],
    title={"text": "❓ Questions Asked"},
    gauge={"axis": {"range": [0, 10]}, "bar": {"color": "#FF9800"}}
))
fig2.update_layout(height=300)
fig2.show()

# --- Longest Monologue ---
display(Markdown(f"### 🎤 Longest Monologue\n⏱️ **{analysis['longest_monologue']}**"))

# --- Sentiment ---
sentiment_colors = {"POSITIVE": "#4CAF50", "NEGATIVE": "#F44336", "NEUTRAL": "#9E9E9E"}
fig3 = go.Figure(go.Indicator(
    mode="gauge+number+delta",
    value=1 if analysis["call_sentiment"] == "POSITIVE" else (0 if analysis["call_sentiment"] == "NEGATIVE" else 0.5),
    title={"text": f"💡 Sentiment: {analysis['call_sentiment']}"},
    gauge={
        "axis": {"range": [0, 1]},
        "bar": {"color": sentiment_colors.get(analysis["call_sentiment"].upper(), "#2196F3")},
        "steps": [
            {"range": [0, 0.33], "color": "#F44336"},
            {"range": [0.33, 0.66], "color": "#9E9E9E"},
            {"range": [0.66, 1], "color": "#4CAF50"},
        ],
    }
))
fig3.update_layout(height=300)
fig3.show()

# --- Actionable Insight ---
display(Markdown("## 📌 Actionable Insight"))
display(Markdown(f"👉 **{analysis['actionable_insight']}**"))


## 🎧 Sales Call Analysis Dashboard

---

### 🎤 Longest Monologue
⏱️ **0:00:11 (sales_rep)**

## 📌 Actionable Insight

👉 **Customer mentioned pricing — follow up with a proposal.**